# Action1： Santa的接待安排
## 1.数据加载

In [1]:
import pandas as pd
import numpy as np
santa = pd.read_csv('./family_data.csv',index_col=0)
santa

,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5,choice_6,choice_7,choice_8,choice_9,n_people
family_id,,,,,,,,,,,
0,52,38,12,82,33,75,64,76,10,28,4
1,26,4,82,5,11,47,38,6,66,61,4
2,100,54,25,12,27,82,10,89,80,33,3
3,2,95,1,96,32,6,40,31,9,59,2
4,53,1,47,93,26,3,46,16,42,39,4
...,...,...,...,...,...,...,...,...,...,...,...
4995,16,1,66,33,18,70,56,46,86,60,4
4996,88,66,20,17,26,54,81,91,59,48,2
4997,32,66,54,17,27,21,74,81,3,7,6


## 2.数据预处理
* 1）计算Perference Cost矩阵 pcost_mat
* 2）计算Accounting Cost矩阵 acost_mat
* 3）计算每个家庭的人数 FAMILY_SIZE
* 4）每个家庭的倾向选择（choice_） DESIRED


In [2]:
#  定义接待安排矩阵preference cost，代表Santa的个性化安排能力
# n：家庭 ， choice：提前时间（天）
def get_penalty(n,choice): 
    penalty = None
    if choice == 0 :
        penalty = 0
    elif choice == 1 :
        penalty = 50
    elif choice == 2 :
        penalty = 50+9*n
    elif choice == 2 :
        penalty = 50+9*n  
    elif choice == 3 :
        penalty = 100+9*n
    elif choice == 4 :
        penalty = 200+9*n
    elif choice == 5 :
        penalty = 200+18*n
    elif choice == 6 :
        penalty = 300+18*n
    elif choice == 7 :
        penalty = 300+36*n
    elif choice == 8 :
        penalty = 400+26*n
    elif choice == 9 :
        penalty = 500+(36+199)*n
    else :
        penalty = 500+(36+398)*n
    return penalty


In [3]:
DAYS = 100 #安排的天数
FAMILYS = 5000 # 所有家庭数
MIN_OCCYPANCY = 125 # 每天访问人数下限
MAX_OCCYPANCY = 300 # 每天访问人数上限

# 1）计算Perference Cost矩阵 pcost_mat
# 计算pcost_mat ,个性化安排能力，即每个家庭在1-100天访问时的pentaly
pcost_mat = np.full(shape=(FAMILYS,100),fill_value=99999) 

for f in range(FAMILYS):
    f_num = santa.loc[f,'n_people']
    # 对第f个家庭，初始化 pcost = other choice的penalty
    pcost_mat[f,:] = get_penalty(f_num,10)
    # choice0-9的penalty
    for choice in range(10):
        temp = santa.loc[f][choice] # 家庭选择的天数
        pcost_mat[f,temp-1] = get_penalty(f_num , choice) # 家庭选择的天数赢得的penalty , -1是为了对应下标
           
pcost_mat


# 值为0表示相符 ， 2236表示家庭成员数为4

array([[2236, 2236, 2236, ..., 2236, 2236, 2236],
       [2236, 2236, 2236, ..., 2236, 2236, 2236],
       [1802, 1802, 1802, ..., 1802, 1802,    0],
       ...,
       [3104, 3104,  556, ..., 3104, 3104, 3104],
       [ 390, 2670, 2670, ..., 2670, 2670, 2670],
       [2236, 2236, 2236, ..., 2236, 2236, 2236]])

In [4]:
# 2）计算Accounting Cost矩阵 acost_mat（前一天的访问人数，当天的访问人数）
# 定义acost_mat ,计算财务成本
acost_mat = np.zeros((MAX_OCCYPANCY+1,MAX_OCCYPANCY+1),dtype=float)

# 当天访问人数
for i in range(acost_mat.shape[0]):
    for j in range(acost_mat.shape[1]): # 前一天访问人数
        diff = abs(i-j)
        acost_mat[i,j] = max(0,(i-125) / 400 * i ** ( 0.5 + diff / 50.0))
acost_mat

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [4.16316072e+15, 3.71482922e+15, 3.31477861e+15, ...,
        7.46610759e+00, 8.36716954e+00, 9.37697794e+00],
       [4.79555148e+15, 4.27883100e+15, 3.81778713e+15, ...,
        8.43020770e+00, 7.52185316e+00, 8.43020770e+00],
       [5.52415954e+15, 4.92860244e+15, 4.39725208e+15, ...,
        9.51970597e+00, 8.49339085e+00, 7.57772228e+00]])

In [5]:
# 3）计算每个家庭的人数 FAMILY_SIZE
# FAMILY_MEMBER ：家庭成员数量
FAMILY_MEMBER = santa['n_people'].values
FAMILY_MEMBER

array([4, 4, 3, ..., 6, 5, 4])

In [6]:

# 4）每个家庭的倾向选择（choice_） DESIRED
# DESIRED ： 各家庭选择的天数对应list下标
DESIRED = santa.values[:,:-1]-1
DESIRED

array([[51, 37, 11, ..., 75,  9, 27],
       [25,  3, 81, ...,  5, 65, 60],
       [99, 53, 24, ..., 88, 79, 32],
       ...,
       [31, 65, 53, ..., 80,  2,  6],
       [66, 91,  3, ..., 11, 25, 69],
       [12, 10, 24, ..., 38, 17, 46]])

In [7]:
santa.values[:,:-1]

array([[ 52,  38,  12, ...,  76,  10,  28],
       [ 26,   4,  82, ...,   6,  66,  61],
       [100,  54,  25, ...,  89,  80,  33],
       ...,
       [ 32,  66,  54, ...,  81,   3,   7],
       [ 67,  92,   4, ...,  12,  26,  70],
       [ 13,  11,  25, ...,  39,  18,  47]])

## 3. 使用LP和MIP求解 规划方案
* 1）先使用LP 对绝大部分家庭进行规划
* 2）再使用MIP 对剩余家庭进行规划
* 3）汇总两边的结果 => 最终规划方案


In [8]:
from ortools.linear_solver import pywraplp

#  定义线性规划
def sloveLP():
    # 使用线性规划优化器
    solver = pywraplp.Solver('AssignmentProblem',pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
    # 使用整数优化器
#     solver = pywraplp.Solver('AssignmentProblem', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

    x = {} # 决策变量：family 再第j天是否参观
    
    # 每天参观的family
    candidates = [[] for x in range(DAYS)]
    
    for i in range(FAMILYS):  # family
        for j in DESIRED[i,:]: # family的choice（前多少天参观）
            candidates[j].append(i) # 在第j天参观的family
            # 定义决策变量x[i,j] , i代表family，j代表days
            x[i,j] = solver.BoolVar('x[%i,%i]'%(i,j))
#     print(candidates)

    # 计算每天参观的人数       # 是否参观  #family成员数量         # 每个family              # 1-100天
    day_occupancy = [solver.Sum([x[i,j] * FAMILY_MEMBER[i] for i in candidates[j]]) for j in range(DAYS)]
#     print(day_occupancy)
    
    # 每个family的10个choice总计     # sum         # choice              # family 
    family_presence = [solver.Sum(x[i,j] for j in DESIRED[i,:]) for i in range(FAMILYS)]
    
    # 定义目标函数 preference_cost 
    preference_cost = solver.Sum([pcost_mat[i,j] * x[i,j] for i in range(FAMILYS) for j in DESIRED[i,:]])
    #满足preference_cost最小原则
    solver.Minimize(preference_cost)
    
    # 增加约束条件1 , 当前人数不少于前一天人数 +-30
    for j in range(DAYS-1):
        solver.Add(day_occupancy[j]-day_occupancy[j+1]<=30)
        solver.Add(day_occupancy[j+1]-day_occupancy[j]<=30)
        
    # 增加约束条件2，每个family都在10个choice中出现1次
    for i in range(FAMILYS):
        solver.Add(family_presence[i]==1)
        
    # 增加约束条件3,每天人数再125-300之间
    for i in range(DAYS):
        solver.Add(day_occupancy[i]>=MIN_OCCYPANCY)
        solver.Add(day_occupancy[i]<=MAX_OCCYPANCY)
        
    result = solver.Solve()
    # 计算结果
    print(solver.Objective().Value())
    
    # 参观日期的安排    
    temp = [(i,j,x[i,j].solution_value()) for i in range(FAMILYS) for j in DESIRED[i,:] if x[i,j].solution_value()>0] 
    df = pd.DataFrame(temp,columns=['family_id','day','result'])
    
    return df

In [9]:
%%time
result = sloveLP()
result

70746.65476190476
CPU times: user 19.4 s, sys: 166 ms, total: 19.6 s
Wall time: 32.6 s


,family_id,day,result
0,0,51,1.0
1,1,25,1.0
2,2,99,1.0
3,3,1,1.0
4,4,52,1.0
...,...,...,...
5067,4995,15,1.0
5068,4996,87,1.0
5069,4997,31,1.0
5070,4998,91,1.0


In [10]:
result['result'].value_counts()

1.00    4250
1.00     679
0.25       2
0.80       2
0.75       2
        ... 
0.20       1
0.80       1
0.60       1
0.20       1
0.75       1
Name: result, Length: 142, dtype: int64

In [11]:
# 已经安排上的family
assigned_df = result[result.result > 0.99]
assigned_df

,family_id,day,result
0,0,51,1.0
1,1,25,1.0
2,2,99,1.0
3,3,1,1.0
4,4,52,1.0
...,...,...,...
5067,4995,15,1.0
5068,4996,87,1.0
5069,4997,31,1.0
5070,4998,91,1.0


## 已安排了4937个family，接下来使用MIP对剩余的family进行安排

In [12]:
# 还未安排上的family
unassigned_df = result[(result.result < 0.99)&(result.result>1-0.99)]
unassigned_df

,family_id,day,result
69,69,4,0.500000
70,69,21,0.500000
323,322,81,0.250000
324,322,56,0.750000
346,344,22,0.428571
...,...,...,...
4490,4421,69,0.500000
4956,4886,27,0.250000
4957,4886,98,0.750000
4982,4911,25,0.428571


In [13]:
# 对未完成规划的family进行规划
# 计算每天访问的人数
assigned_df['family_size'] = FAMILY_MEMBER[assigned_df.family_id]
assigned_df

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,family_id,day,result,family_size
0,0,51,1.0,4
1,1,25,1.0,4
2,2,99,1.0,3
3,3,1,1.0,2
4,4,52,1.0,4
...,...,...,...,...
5067,4995,15,1.0,4
5068,4996,87,1.0,2
5069,4997,31,1.0,6
5070,4998,91,1.0,5


In [14]:
# 按照day进行聚合
occupancy = assigned_df.groupby('day').family_size.sum().values
occupancy

array([289, 265, 295, 298, 274, 248, 202, 226, 265, 297, 293, 291, 266,
       241, 238, 262, 295, 300, 271, 238, 205, 233, 260, 296, 298, 296,
       268, 237, 234, 249, 283, 295, 265, 231, 205, 172, 195, 233, 266,
       242, 214, 181, 199, 229, 255, 289, 256, 227, 200, 194, 222, 248,
       267, 241, 208, 177, 169, 206, 238, 213, 180, 152, 124, 121, 146,
       170, 189, 158, 131, 123, 125, 130, 162, 189, 160, 131, 121, 125,
       138, 167, 194, 167, 136, 120, 120, 155, 184, 211, 181, 152, 122,
       124, 126, 158, 183, 151, 128, 123, 122, 124])

In [15]:
min_occupancy = np.array([max(0,MIN_OCCYPANCY-i) for i in occupancy])
max_occupancy = np.array([MAX_OCCYPANCY-i for i in occupancy])
max_occupancy

array([ 11,  35,   5,   2,  26,  52,  98,  74,  35,   3,   7,   9,  34,
        59,  62,  38,   5,   0,  29,  62,  95,  67,  40,   4,   2,   4,
        32,  63,  66,  51,  17,   5,  35,  69,  95, 128, 105,  67,  34,
        58,  86, 119, 101,  71,  45,  11,  44,  73, 100, 106,  78,  52,
        33,  59,  92, 123, 131,  94,  62,  87, 120, 148, 176, 179, 154,
       130, 111, 142, 169, 177, 175, 170, 138, 111, 140, 169, 179, 175,
       162, 133, 106, 133, 164, 180, 180, 145, 116,  89, 119, 148, 178,
       176, 174, 142, 117, 149, 172, 177, 178, 176])

In [16]:
# 对整数规划问题（MIP）进行求解

def solveMIP(family, min_occupancy, max_occupancy):
    solver = pywraplp.Solver('AssignmentProblem', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    x = {} # 决策变量：family 再第j天是否参观
    n_families=len(family)
    
    # 每天参观的family
    candidates = [[] for x in range(DAYS)]
    
    for i in family:  # family
        for j in DESIRED[i,:]: # family的choice（前多少天参观）
            candidates[j].append(i) # 在第j天参观的family
            # 定义决策变量x[i,j] , i代表family，j代表days
            x[i,j] = solver.BoolVar('x[%i,%i]'%(i,j))
#     print(candidates)

    # 计算每天参观的人数       # 是否参观  #family成员数量         # 每个family              # 1-100天
    day_occupancy = [solver.Sum([x[i,j] * FAMILY_MEMBER[i] for i in candidates[j]]) for j in range(DAYS)]
#     print(day_occupancy)
    
    # 每个family的10个choice总计     # sum         # choice              # family 
    family_presence = [solver.Sum(x[i,j] for j in DESIRED[i,:]) for i in family]
    
    # 定义目标函数 preference_cost 
    preference_cost = solver.Sum([pcost_mat[i,j] * x[i,j] for i in family for j in DESIRED[i,:]])
    #满足preference_cost最小原则
    solver.Minimize(preference_cost)
    
        
    # 每个family都在10个choice中出现1次
    for i in range(n_families):
        solver.Add(family_presence[i]==1)
        
    # 每天人数再125-300之间
    for i in range(DAYS):
        solver.Add(day_occupancy[i]>=min_occupancy[i])
        solver.Add(day_occupancy[i]<=max_occupancy[i])
        
    result = solver.Solve()
    # 计算结果
    print(solver.Objective().Value())
    
    # 参观日期的安排    
    temp = [(i,j,x[i,j].solution_value()) for i in family for j in DESIRED[i,:] if x[i,j].solution_value()>0] 
    df = pd.DataFrame(temp,columns=['family_id','day','result'])
    
    return df

In [17]:
%%time
unassigned = unassigned_df.family_id.unique()
result = solveMIP(unassigned, min_occupancy, max_occupancy)
result

1927.0
CPU times: user 99.8 ms, sys: 982 µs, total: 101 ms
Wall time: 150 ms


,family_id,day,result
0,69,4,1.0
1,322,81,1.0
2,344,22,1.0
3,357,54,1.0
4,455,0,1.0
...,...,...,...
60,4404,7,1.0
61,4414,80,1.0
62,4421,69,1.0
63,4886,98,1.0


In [18]:
# 将所得结果合并
df = pd.concat((assigned_df[['family_id','day']],result[['family_id','day']])).sort_values('family_id')
df

,family_id,day
0,0,51
1,1,25
2,2,99
3,3,1
4,4,52
...,...,...
5067,4995,15
5068,4996,87
5069,4997,31
5070,4998,91


In [19]:
from numba import njit

# 根据安排的情况，重新计算这个安排的preference cost
@njit(fastmath=True)
def pcost(prediction):
    day_occuracy = np.zeros(DAYS+1 , dtype=np.int64)
    penalty = 0
    for (i,p) in enumerate(prediction):
        # 计算家庭人数
        n = FAMILY_MEMBER[i]
        # 第i个家庭，p天访问时的cost
        penalty += pcost_mat[i,p]
        # 计算当天人数
        day_occuracy[p] += n
    
    return penalty,day_occuracy

# 根据安排的情况，计算这个安排的accounting cost
@njit(fastmath=True)
def acost(day_occuracy):
    accounting_cost = 0
    num_out_range = 0
    day_occuracy[-1] = day_occuracy[-2]
    for day in range(DAYS):
        n_pl = day_occuracy[day+1] # 前一天的财务成本
        n = day_occuracy[day] # 今天的财务成本

        # 如果超过了承载范围则设置out range
        num_out_range += (n>MAX_OCCYPANCY) or (n<MIN_OCCYPANCY)
        # 计算accounting cost
        accounting_cost += acost_mat[n,n_pl]
    
    return accounting_cost , num_out_range

# 计算prediction
@njit(fastmath=True)
def cost_prediction(prediction):
    # 基于prediction计算 preference cost 和 accounting cost
    penalty , day_occuracy = pcost(prediction) # preference cost和每天承载数量
    accounting_cost , num_out_range = acost(day_occuracy) # 根据每天承载数量计算accounting cost
    final_score = penalty + accounting_cost + num_out_range * 999999 #防止代价越界
    return final_score

In [20]:
prediction = df.day.values
print(cost_prediction(prediction))

78892.00658437214


In [21]:
def save_result(pred,filename):
    result = pd.DataFrame(range(FAMILYS),columns=['family_id'])
    result['assigned_day'] = pred+1
    result.to_csv(filename , index=False)
    print(filename+' saved')
    return result

result = save_result(prediction,'submission.csv')

submission.csv saved


## 对得到的解决方案进行优化

In [22]:
def find_better(pred):
    fobs = np.argsort(FAMILY_MEMBER) #打乱返回随机下标
    
    score = cost_prediction(pred)
    original_score = np.inf # 打擂法(穷举)
    
    # 如果找不到更新就退出
    while score < original_score:
        original_score = score
        
        for family_id in fobs:
            for pick in range(10):
                # 得到family_id 在choice_pick 的日期day
                day = DESIRED[family_id, pick]
                # family_id原有日期
                old_value = pred[family_id]
                #将原有old_value替换为现在的choice pick
                pred[family_id] = day
                #重新计算分数
                new_score = cost_prediction(pred)
                # 如果分数比原来小则更换为新的，否则换为原来的分数
                if new_score < score:
                    score = new_score
                else:
                    pred[family_id] = old_value
        print(score,end='\r')
    print(score)

In [23]:
new = prediction.copy()
find_better(new)

73629.94368126492


In [25]:
save_result(new , 'submission2.csv')

submission2.csv saved


,family_id,assigned_day
0,0,52
1,1,26
2,2,100
3,3,2
4,4,53
...,...,...
4995,4995,16
4996,4996,88
4997,4997,32
4998,4998,92


### 优化后结果比之前提高了不少，由78892提升至了73629